## Assign Scores - Universities

This worksheet is designed to give some sense of how well individual universities fit in a tier.
This is done by counting up the placements (and hires) of a university, computing the likelihood of those placements (or hires) given the estimated tier placement rates, then creating a score for each univerity by taking a weighted sum of the placements with the weights being equal to the estimated mean value of offers from those universities.  The value of hires is a weighted sum of all hires with the weights be equal to the estimated value of the graduates hired.

There is a bias in this method because tier 1 is oversampled.  All its values will be higher just because we were more likely to find its placements.

In [1]:
using HTTP, JSON, PrettyTables, JLD, DotEnv, Distributions, LinearAlgebra, Dates, MySQL
cfg = DotEnv.config("../.env")
files_path = cfg["files_path"]
#adjacency = load(files_path*"placement_rates.jld")["placement_rates"]
classification_properties = load(files_path*"classification_properties.jld")["classification_properties"]
rep = JSON.parsefile(files_path*"likelihood_ratios.json");
refresh_mysql_db = false
classification_properties

Dict{String, Any} with 7 entries:
  "data_loaded"              => DateTime("2024-08-31T23:13:46.046")
  "row_labels"               => Any["TYPE 1 (20 insts)", "TYPE 2 (62 insts)", "…
  "unmatched_index"          => 10
  "number_of_academic_types" => 5
  "institution_counts"       => [20, 62, 179, 352, 568, 159, 255, 635, 443, 1, …
  "algorithm_run_id"         => 6
  "num_years"                => 24

In [2]:
function value_quantiles(a, value)
    # a is a vector of variable length, value is a real number
    # returns the proportion of values in the array that are less than or equal to value
    # dists[t][:,1] is the array of hire values for each member of tier t
    # dists[t][:,2] is the array of placement values for each member of tier t
       return length(a[a .<= value])/length(a)
end


value_quantiles (generic function with 1 method)

In [3]:
# creates an array conntaining matrices of different sizes
# dists[t][:,1] is an array of values of hires by each member of tier t
# dists[t][:,2] is an array of values of the placements of each member of the tier
dists = Array{Any}(nothing,5)
for k in 1:classification_properties["number_of_academic_types"]
    dist = zeros(classification_properties["institution_counts"][k],2)
    n = 1
    for r in rep
        if r["tier"]==k
            dist[n] = r["hiring_value"]
            dist[n+classification_properties["institution_counts"][k]] = r["placement_value"]
            n += 1
        end
    end
    dists[k] = dist
end

LoadError: BoundsError: attempt to access 179×2 Matrix{Float64} at index [359]

In [4]:
#length(a[a .< x])/length(a)
function distribution(a)
    b = Any[]
    for (index,value) in enumerate(a)
        push!(b,length(a[a .<= value])/length(a))
    end
    return b
end
            

distribution (generic function with 1 method)

In [5]:
# display hiring values for each tier
using Plots
y = distribution(sort(dists[1][:,1]))
x = sort(dists[1][:,1])
plot(x,y)
y = distribution(sort(dists[2][:,1]))
x = sort(dists[2][:,1])
plot!(x,y)
y = distribution(sort(dists[3][:,1]))
x = sort(dists[3][:,1])
plot!(x,y)
y = distribution(sort(dists[4][:,1]))
x = sort(dists[4][:,1])
plot!(x,y)
y = distribution(sort(dists[5][:,1]))
x = sort(dists[5][:,1])
plot!(x,y)
            


LoadError: MethodError: no method matching getindex(::Nothing, ::Colon, ::Int64)

In [6]:
# display offer values for each tier
y = distribution(sort(dists[1][:,2]))
x = sort(dists[1][:,2])
plot(x,y)
y = distribution(sort(dists[2][:,2]))
x = sort(dists[2][:,2])
plot!(x,y)
y = distribution(sort(dists[3][:,2]))
x = sort(dists[3][:,2])
plot!(x,y)
y = distribution(sort(dists[4][:,2]))
x = sort(dists[4][:,2])
plot!(x,y)
y = distribution(sort(dists[5][:,2]))
x = sort(dists[5][:,2])
plot!(x,y)
  

LoadError: MethodError: no method matching getindex(::Nothing, ::Colon, ::Int64)

In [7]:
refresh_mysql_db = false
if refresh_mysql_db == true
    d = DBInterface.connect(MySQL.Connection,cfg["host"], cfg["user"], cfg["password"], 
        db =cfg["database"], port = parse(Int64,cfg["port"]));
    q = """
    delete from t_distribution where algorithm_run_id=?
    """
    delete_statement = DBInterface.prepare(d, q)
    DBInterface.execute(delete_statement,[classification_properties["algorithm_run_id"]])
    
    query = """
        insert into econjobmarket_research.t_distribution (type,institution_id,created,algorithm_run_id, properties)
        values(?,?,?,?,?)
    """
    insert_statement =  DBInterface.prepare(d, query);
    function save_type(insert_statement,t,institution_id,run_id,json_data)
        #values would be the rep dict
        DBInterface.execute(insert_statement, [t,institution_id,now(),run_id,JSON.json(json_data)])
    end

end

In [8]:
new_rep = []
for r in rep
    r["placement_quantile"] = value_quantiles(dists[r["tier"]][:,2],r["placement_value"])
    r["hiring_quantile"] = value_quantiles(dists[r["tier"]][:,1], r["hiring_value"] )
    push!(new_rep, r)
    if refresh_mysql_db == true
        try
            save_type(insert_statement,r["tier"],parse(Int64,r["id"]),
            classification_properties["algorithm_run_id"],r)
        catch e
            rethrow(e)
        end
    end
end
open(files_path*"likelihood_ratios.json", "w") do f
    write(f, JSON.json(new_rep))
end
if refresh_mysql_db == true
    DBInterface.close!(d)
end


LoadError: MethodError: no method matching getindex(::Nothing, ::Colon, ::Int64)

In [9]:
id = 57
for r in new_rep
    if r["id"] == string(id)
        for (key,value) in r
            println(key, " => ", value)
        end
    end
end

In [10]:
value_quantiles(dists[2][:,2], 33)

0.5645161290322581